In [1]:
#Source du code
#https://larevueia.fr/latent-dirichlet-allocation-topic-modeling-en-python/


In [2]:
#Import de pandas
import pandas as pd

In [4]:
#Installation de nltk et de gensim
!pip install nltk
!pip install gensim

  Using cached gensim-4.3.2-cp311-cp311-win_amd64.whl.metadata (8.5 kB)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 1.1 MB/s eta 0:00:22
   ---------------------------------------- 0.1/24.0 MB 1.2 MB/s eta 0:00:20
   ---------------------------------------- 0.3/24.0 MB 1.8 MB/s eta 0:00:14
    --------------------------------------- 0.5/24.0 MB 2.3 MB/s eta 0:00:11
   - -------------------------------------- 0.7/24.0 MB 3.0 MB/s eta 0:00:08
   - -------------------------------------- 1.0/24.0 MB 3.3 MB/s eta 0:00:08
   - -------------------------------------- 1.2/24.0 MB 3.5 MB/s eta 0:00:07
   -- ------------------------------------- 1.2/24.0 MB 3.2 MB/s eta 0:00:08
   -- ------------------------------------- 1.6/24.0 MB 3.5 MB/s eta 0:00:07
   -- -----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires funcy, which is not installed.
pyldavis 3.4.1 requires numexpr, which is not installed.


In [5]:
#Installation de funcy numexpr et pyLDAvis car pyLDAvis 3.4.1 nécessite les packages funcy et numexpr, qui ne sont pas installés.

!pip install funcy numexpr
!pip install pyLDAvis


   ---------------------------------------- 0.0/95.3 kB ? eta -:--:--
   ------------------------- -------------- 61.4/95.3 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 95.3/95.3 kB 1.1 MB/s eta 0:00:00


In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
#Charger le dataset, ici children_covid_analyse.csv
def load_data(path):
    return pd.read_csv(path)
tweets_df=load_data("../../Donnees/analyse_texte/children_covid_analyse.csv") 
#Pour effectuer un autre TM il faudra juste changer le nom du dataset

In [9]:
#La racinisation (stemming) consiste à transformer les mots en forme radicale c’est ce que fait la fonction SnowballStemmer de Nltk.
stemmer = SnowballStemmer('english') #ici anglais car nos tweets sont en anglais

In [11]:
tweets_df['Texte corrigé'].dtype

dtype('O')

In [12]:
#Toutes les lignes de la colonne "Texte corrigé" n'étaient pas toutes de types 'str'
tweets_df['Texte corrigé'] = tweets_df['Texte corrigé'].astype(str)

#Suppression des liens commençant par http et https  
def remove_url(text):
 return re.sub(r'https?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url)
def remove_url1(text):
 return re.sub(r'http?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url1)

In [13]:
#opération de mettre les mots sous leur forme canonique 
#ex: running devient run 
def lemmatize_stemming(text) :
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='n'))

In [14]:
#suppression des stopwords (the, or, and…) et des mots de moins de 3 lettres
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


# Imprimez le résultat intermédiaire pour chaque document
processed_docs = [preprocess(doc) for doc in tweets_df]
for doc in processed_docs:
    # Concaténez les mots pour former une phrase
    doc_text = ' '.join(doc)
    print(preprocess(doc_text))
    



['fichier']
[]
['retweet']
['like']
['répon']
['citat']
['hashtag']
['text']
[]
['phrase']
[]
['sentiment']
['polarité']
['subjectivité']
['phrase', 'nomin']
['text', 'corrigé']
['text', 'tokénizé']


In [17]:
# On stocke les données dans un dictionnaire Gensim et on les convertit au format Bag Of Words(bow).
#On a alors un couple mot/nb d'occurence

# Utilisez la colonne "Texte corrigé" pour le prétraitement
processed_docs = [preprocess(doc) for doc in tweets_df['Texte corrigé']]

# Créez le dictionnaire et le corpus
dictionary = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Créez et entraînez le modèle LDA
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, # Le nombre de topics à extraire
                                       id2word=dictionary, # Le mapping entre les identifiants et les mots
                                       passes=2000) # Le nombre d'itérations dans le corpus/ passages




# Affichez les topics
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)


Topic: 0 -> Words: 0.098*"vaccin" + 0.087*"covid" + 0.074*"child" + 0.016*"kiss" + 0.015*"school" + 0.014*"health" + 0.012*"anaem" + 0.010*"parent" + 0.010*"coronavirus" + 0.009*"move"
Topic: 1 -> Words: 0.069*"vaccin" + 0.059*"covid" + 0.053*"child" + 0.014*"kiss" + 0.012*"health" + 0.011*"india" + 0.010*"age" + 0.010*"anaem" + 0.009*"protect" + 0.008*"long"
Topic: 2 -> Words: 0.044*"vaccin" + 0.042*"child" + 0.036*"covid" + 0.009*"start" + 0.008*"event" + 0.007*"year" + 0.007*"kiss" + 0.006*"novavax" + 0.005*"scienc" + 0.004*"health"
Topic: 3 -> Words: 0.041*"vaccin" + 0.029*"covid" + 0.028*"child" + 0.013*"australia" + 0.008*"tafrishaala" + 0.008*"survey" + 0.006*"news" + 0.006*"coronavirus" + 0.005*"queensland" + 0.005*"crime"
Topic: 4 -> Words: 0.038*"vaccin" + 0.035*"help" + 0.033*"covid" + 0.032*"child" + 0.025*"money" + 0.025*"hopeless" + 0.023*"financ" + 0.023*"covid_" + 0.022*"christian" + 0.021*"philanthropi"
Topic: 5 -> Words: 0.065*"vaccin" + 0.062*"covid" + 0.051*"child" 

temps d'exécution pour 1407 lignes et 1000 passages: 10 min 30.1 s 
pour 2000 passages: 21 min 58.6 s 

In [18]:
#Mesure de la cohérence du modèle
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.41939273995841353


Pour 1000 passages et 10 topics on a une cohérence de 0.40353069757626814
Pour 2000 passages et 10 topics  on a une cohérence de 0.41939273995841353

On observe aucune augmentation significative. 
La cohérence du modèle LDA doit être comprise entre 0.4 et 0.7. Dans la documentation il est proposé d'augmenter le nb de passages à 10000 pour avoir un score proche de 70%


In [19]:
#On garde les résultats dans un df
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

In [20]:
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7: "Topic 8", 8: "Topic 9", 9: "Topic 10"})

,0,1,2,3,4,5,6,7,8,9
Topic 1,"(0.098, vaccin)","(0.087, covid)","(0.074, child)","(0.016, kiss)","(0.015, school)","(0.014, health)","(0.012, anaem)","(0.010, parent)","(0.010, coronavirus)","(0.009, move)"
Topic 2,"(0.069, vaccin)","(0.059, covid)","(0.053, child)","(0.014, kiss)","(0.012, health)","(0.011, india)","(0.010, age)","(0.010, anaem)","(0.009, protect)","(0.008, long)"
Topic 3,"(0.044, vaccin)","(0.042, child)","(0.036, covid)","(0.009, start)","(0.008, event)","(0.007, year)","(0.007, kiss)","(0.006, novavax)","(0.005, scienc)","(0.004, health)"
Topic 4,"(0.041, vaccin)","(0.029, covid)","(0.028, child)","(0.013, australia)","(0.008, tafrishaala)","(0.008, survey)","(0.006, news)","(0.006, coronavirus)","(0.005, queensland)","(0.005, crime)"
Topic 5,"(0.038, vaccin)","(0.035, help)","(0.033, covid)","(0.032, child)","(0.025, money)","(0.025, hopeless)","(0.023, financ)","(0.023, covid_)","(0.022, christian)","(0.021, philanthropi)"
Topic 6,"(0.065, vaccin)","(0.062, covid)","(0.051, child)","(0.016, question)","(0.014, share)","(0.013, fear)","(0.013, watch)","(0.012, pidsocieti)","(0.012, idsainfo)","(0.012, realtimecovid)"
Topic 7,"(0.049, vaccin)","(0.049, covid)","(0.046, child)","(0.039, covaxin)","(0.009, bharatbiotech)","(0.008, variant)","(0.007, administ)","(0.007, cnbcnow)","(0.007, endors)","(0.007, aiim)"
Topic 8,"(0.042, covid)","(0.038, vaccin)","(0.034, child)","(0.022, israel)","(0.017, scienc)","(0.013, ontario)","(0.013, australia)","(0.013, albert)","(0.013, corrupt)","(0.012, murder)"
Topic 9,"(0.112, vaccin)","(0.090, child)","(0.087, covid)","(0.037, finer)","(0.023, age)","(0.020, year)","(0.017, approv)","(0.013, modern)","(0.012, biontech)","(0.011, author)"
Topic 10,"(0.104, vaccin)","(0.074, child)","(0.070, covid)","(0.030, kiss)","(0.025, coronavirus)","(0.015, parent)","(0.015, finer)","(0.012, health)","(0.010, news)","(0.009, move)"


In [21]:
import pyLDAvis.gensim_models

In [22]:
#Visu interractive affichant les topics, les mots etc
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.090848 -0.033087       1        1  27.305277
8     -0.181994 -0.025989       2        1  26.425650
9     -0.126526 -0.054107       3        1  14.833830
1     -0.037297  0.011106       4        1   7.303474
5      0.026508  0.068389       5        1   6.300066
6      0.043739  0.076284       6        1   4.550385
4      0.155413 -0.189540       7        1   4.170647
7      0.078069  0.054793       8        1   3.617928
2      0.053587  0.056274       9        1   3.243301
3      0.079349  0.035878      10        1   2.249443, topic_info=          Term         Freq        Total Category  logprob  loglift
15      vaccin  2018.000000  2018.000000  Default  30.0000  30.0000
3        covid  1652.000000  1652.000000  Default  29.0000  29.0000
1        child  1566.000000  1566.000000  Default  28.0000  28.0000
47       finer   320.000000   320.000000  Default  27.0000  27.0000
17         age   183.000000   183.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
263   lockdown     1.539267    23.500238  Topic10  -5.8098   1.0688
46      approv     1.509766   128.151390  Topic10  -5.8292  -0.6468
261      india     1.494708    57.421344  Topic10  -5.8392   0.1460
68      school     1.454589   137.600656  Topic10  -5.8664  -0.7551
110  communiti     1.452521    38.616985  Topic10  -5.8678   0.5141

[611 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1647      4  0.893809     abcnew
825       1  0.111137     access
825       2  0.222273     access
825       5  0.222273     access
825       8  0.333410     access
...     ...       ...        ...
1165      1  0.806035  youngster
1165      7  0.080604  youngster
1165      9  0.080604  youngster
1756      4  0.792100      zycov
1569      4  0.792097     zycovd

[1056 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 9, 10, 2, 6, 7, 5, 8, 3, 4])